In [ ]:
import os
from google.colab import files
uploaded = files.upload()

Saving from_model.csv to from_model.csv


In [85]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from community import community_louvain
import numpy as np

In [86]:
df = pd.read_csv('/content/from_model_v2_22.csv', encoding="utf-8")
df

,Unnamed: 0,username,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,tweet,verdict,RT,source_user,target_user
0,0,PIACaraga,2021-09-21 23:53:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,"TINGNAN: Kahapon, Setyembre 20, inilabas ni @n...",TRUE,True,pcoogov,PIACaraga
1,1,PIACaraga,2021-09-21 23:53:27+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,PIACaraga
2,2,PIACaraga,2021-09-21 23:52:23+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,"PANOORIN: Kahapon, Setyembre 20, nagbigay ng u...",TRUE,True,pcoogov,PIACaraga
3,3,YamauchiElena,2021-09-21 20:45:11+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,YamauchiElena
4,4,greg_kassie,2021-09-21 16:28:08+00:00,,1.440000e+18,tl,0.0,0.0,0.0,34.0,Dapat mayroong sapat na abiso sa mga lugar na ...,MOSTLY_TRUE,True,WinGatchalian74,greg_kassie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4813,4813,roysant51162134,2021-09-17 08:29:20+00:00,,1.440000e+18,tl,0.0,0.0,1.0,0.0,@EdsonCGuido @ABSCBNNews habang dumadami nabab...,MOSTLY_FALSE,False,roysant51162134,ABSCBNNews
4814,4814,dzrhnews,2021-09-17 04:23:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,Pakiusap na payagan na rin ang mga naka-unang ...,TRUE,False,dzrhnews,liezel_once
4815,4815,GrowlHarhar,2021-09-17 03:09:24+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,@CityMouse14 @inquirerdotnet @KAguilarINQ Ikaw...,MOSTLY_TRUE,False,GrowlHarhar,inquirerdotnet
4816,4816,GrowlHarhar,2021-09-17 03:09:24+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,@CityMouse14 @inquirerdotnet @KAguilarINQ Ikaw...,MOSTLY_TRUE,False,GrowlHarhar,KAguilarINQ


### Perform separation of Nodes and Edges

In [87]:
G = nx.from_pandas_edgelist(df, 'source_user', 'target_user') #Turn df into graph
G_sorted = pd.DataFrame(sorted(G.degree, key=lambda x: x[1], reverse=True))
G_sorted.columns = ['nconst','degree']
G_sorted.head()

,nconst,degree
0,ABSCBNNews,439
1,NaN,318
2,TVPatrol,79
3,dzbb,56
4,PhilippineStar,55


In [88]:
G.remove_edges_from(nx.selfloop_edges(G))
G_tmp = nx.k_core(G, 0) #Exclude nodes with degree less than 10
G_tmp.number_of_nodes()

1786

In [89]:
partition = community_louvain.best_partition(G_tmp)
#Turn partition into dataframe
partition1 = pd.DataFrame([partition]).T
partition1 = partition1.reset_index()
partition1.columns = ['names','group']

In [90]:
G_sorted = pd.DataFrame(G_tmp.degree)
G_sorted.columns = ['names','degree']
G_sorted.head()
dc = G_sorted

### Visualization (Broken)

In [91]:
#pos = nx.spring_layout(G_tmp)
#f, ax = plt.subplots(figsize=(10, 10))
#plt.style.use('ggplot')
#cc = nx.betweenness_centrality(G2)
#nodes = nx.draw_networkx_nodes(G_tmp, pos,
#                               cmap=plt.cm.Set1,
#                               node_color=combined['group'],
#                               alpha=0.8)
#nodes.set_edgecolor('k')
#nx.draw_networkx_labels(G_tmp, pos, font_size=8)
#nx.draw_networkx_edges(G_tmp, pos, width=1.0, alpha=0.2)
#plt.savefig('twitterFollowers.png')

### Output Nodes and Edges

In [92]:
combined = pd.merge(dc,partition1, how='left', left_on="names",right_on="names")

In [93]:
combined = combined.rename(columns={"names": "Id"}) #I've found Gephi really likes when your node column is called 'Id'
edges = nx.to_pandas_edgelist(G_tmp)
nodes = combined['Id']

### Adding reference to the row index of tweets

In [94]:
#Name
int(combined.loc[combined.Id=='pcoogov'].index[0])
#Null
combined.loc[combined.Id.isnull()]

,Id,degree,group
19,NaN,318,4


In [95]:
combined['Label'] = combined['Id']
combined['source'] = ''
combined['target'] = ''
combined

,Id,degree,group,Label,source,target
0,pcoogov,48,5,pcoogov,,
1,PIACaraga,8,5,PIACaraga,,
2,YamauchiElena,1,5,YamauchiElena,,
3,WinGatchalian74,52,2,WinGatchalian74,,
4,greg_kassie,1,2,greg_kassie,,
...,...,...,...,...,...,...
1781,haydeesampang,1,32,haydeesampang,,
1782,alejand12072264,1,58,alejand12072264,,
1783,liezel_once,1,7,liezel_once,,
1784,KAguilarINQ,1,3,KAguilarINQ,,


In [96]:
for i in combined.index:
  combined.at[i,'source'] = []
  combined.at[i,'target'] = []

In [83]:
for i in df.index:
  row = df.iloc[i]
  source = row.source_user
  target = row.target_user
  if source is not np.nan:
    combined_row = combined.loc[combined.Id==source]
  else:
    combined_row = combined.loc[combined.Id.isnull()]

  some_list = combined_row.source[combined_row.index[0]]
  some_list = some_list + [i]
  combined.at[combined_row.index[0],'source'] = some_list
  

  if target is not np.nan:
    combined_row = combined.loc[combined.Id==target]
  else:
    combined_row = combined.loc[combined.Id.isnull()]


  some_list = combined_row.target[combined_row.index[0]]
  some_list = some_list + [i]
  combined.at[combined_row.index[0],'target'] = some_list
  


combined

,Id,degree,group,Label,source,target
0,pcoogov,48,0,pcoogov,"[0, 1, 2, 3, 12, 13, 22, 26, 28, 29, 50, 51, 5...","[1296, 1400, 4690, 4725]"
1,PIACaraga,8,0,PIACaraga,"[88, 89, 90, 144, 145, 146, 147, 148, 149, 150...","[0, 1, 2, 91, 92, 162, 174, 175, 506, 614, 706..."
2,YamauchiElena,1,0,YamauchiElena,[],"[3, 2420]"
3,WinGatchalian74,52,2,WinGatchalian74,"[4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 17, 18, 19,...","[1366, 2700, 3770, 4810]"
4,greg_kassie,1,2,greg_kassie,[],"[4, 215, 251, 252, 408, 1367, 1997, 2701, 3269..."
...,...,...,...,...,...,...
1781,haydeesampang,1,31,haydeesampang,[],[4797]
1782,alejand12072264,1,57,alejand12072264,[],[4798]
1783,liezel_once,1,7,liezel_once,[],[4814]
1784,KAguilarINQ,1,8,KAguilarINQ,[],[4816]


### Export to csv

In [84]:
edges.to_csv("edges.csv")
combined.to_csv("nodes.csv")